In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
import evaluate

/home/mehedi/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
main_data = pd.read_csv("dataset/training.csv",names=['id','name', 'label', 'sentence'])
main_data.drop(index=0)
main_data = main_data.dropna()
drop_columns = ["id", "name"]
main_data = main_data.drop(drop_columns, axis=1)
main_data  = main_data.iloc[:1000,:]

# md_data = md_data.head(10000)

In [ ]:
def encoder(data):
    values_list = list(data.unique())
    temp_dict = {}
    for item in values_list:
        index = values_list.index(item)
        temp_dict.update({item:index})
    print(temp_dict)
    for d in data.index:
        data[d] = temp_dict[data[d]]
    return data, temp_dict

print(main_data['label'].value_counts())
classes = main_data["label"].unique()

main_data['label'], label_map = encoder(main_data['label'])

Positive      425
Neutral       279
Negative      194
Irrelevant    102
Name: label, dtype: int64
{'Positive': 0, 'Neutral': 1, 'Negative': 2, 'Irrelevant': 3}


In [ ]:
# data = pd.read_csv("dataset/training_updated.csv")
train_data = main_data.sample(frac=0.8, random_state=42)
test_data = main_data.drop(train_data.index)
hg_train_data = Dataset.from_pandas(train_data)
hg_test_data = Dataset.from_pandas(test_data)
# print(f'The training dataset has {len(train_data)} records.')
# print(f'The testing dataset has {len(test_data)} records.')
print(f'The length of train_data is {len(hg_train_data)}')
print(f'The length of test_data is {len(hg_test_data)}')

The length of train_data is 800
The length of test_data is 200


In [ ]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
# tokenizer = AutoTokenizer.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

In [ ]:
# Funtion to tokenize data
def tokenize_dataset(data):
    return tokenizer(data["sentence"],
                     truncation=True,
                     padding="max_length")

dataset_train = hg_train_data.map(tokenize_dataset)
dataset_test = hg_test_data.map(tokenize_dataset)

Map: 100%|██████████| 200/200 [00:00<00:00, 5344.66 examples/s]


In [ ]:
# Load model
# model = AutoModelForSequenceClassification.from_pretrained("j-hartmann/emotion-english-distilroberta-base", num_labels=len(classes), ignore_mismatched_sizes=True).to(device=device)
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english", num_labels=len(classes), ignore_mismatched_sizes=True).to(device=device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./CA/",
    logging_dir='.CA/logs',
    logging_strategy='epoch',
    logging_steps=10,
    num_train_epochs=100,
    per_device_train_batch_size=20,
    per_device_eval_batch_size=20,
    learning_rate=5e-6,
    seed=42,
    # save_strategy='epoch',
    # save_steps=100,
    evaluation_strategy='epoch',
    eval_steps=100,
    # load_best_model_at_end=True
)

In [ ]:
def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    # probabilities = tf.nn.softmax(logits)
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)
# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics,
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

In [ ]:
trainer.train()

/home/mehedi/anaconda3/envs/text-env/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  1%|          | 40/4000 [00:35<58:07,  1.14it/s] 

{'loss': 1.2421, 'learning_rate': 4.95e-06, 'epoch': 1.0}


                                                 
  1%|          | 40/4000 [00:40<58:07,  1.14it/s]

{'eval_loss': 1.2327674627304077, 'eval_accuracy': 0.43, 'eval_runtime': 4.9211, 'eval_samples_per_second': 40.642, 'eval_steps_per_second': 2.032, 'epoch': 1.0}


  2%|▏         | 80/4000 [01:15<57:01,  1.15it/s]  

...

100%|██████████| 4000/4000 [1:05:30<00:00,  1.22it/s]

{'loss': 0.0117, 'learning_rate': 0.0, 'epoch': 100.0}


                                                     
100%|██████████| 4000/4000 [1:05:35<00:00,  1.02it/s]

{'eval_loss': 0.31876227259635925, 'eval_accuracy': 0.93, 'eval_runtime': 4.7831, 'eval_samples_per_second': 41.814, 'eval_steps_per_second': 2.091, 'epoch': 100.0}
{'train_runtime': 3935.3272, 'train_samples_per_second': 20.329, 'train_steps_per_second': 1.016, 'train_loss': 0.10648007635772229, 'epoch': 100.0}


TrainOutput(global_step=4000, training_loss=0.10648007635772229, metrics={'train_runtime': 3935.3272, 'train_samples_per_second': 20.329, 'train_steps_per_second': 1.016, 'train_loss': 0.10648007635772229, 'epoch': 100.0})

In [ ]:
# Trainer evaluate
trainer.evaluate(dataset_test)

100%|██████████| 10/10 [00:04<00:00,  2.12it/s]


{'eval_loss': 0.31876227259635925,
 'eval_accuracy': 0.93,
 'eval_runtime': 5.0785,
 'eval_samples_per_second': 39.382,
 'eval_steps_per_second': 1.969,
 'epoch': 100.0}

In [ ]:
from scipy.special import softmax
y_test_predict = trainer.predict(dataset_test)
y_test_logits = y_test_predict.predictions
y_test_probabilities = softmax(y_test_logits, axis=1)
y_test_pred_labels = np.argmax(y_test_probabilities, axis=1)
y_test_actual_labels = y_test_predict.label_ids
metric_f1 = evaluate.load("f1")
f1_score = metric_f1.compute(predictions=y_test_pred_labels, references=y_test_actual_labels, average='micro')
metric_recall = evaluate.load("recall")
recall_score = metric_recall.compute(predictions=y_test_pred_labels, references=y_test_actual_labels, average='micro')

print(f"""
        Recall Score ----> {recall_score.values()}
        F1 Score --------> {f1_score.values()}

    """)

100%|██████████| 10/10 [00:04<00:00,  2.15it/s]



        Recall Score ----> dict_values([0.93])
        F1 Score --------> dict_values([0.93])

    


In [ ]:
folder_name = "analysis"
tokenizer.save_pretrained('./'+folder_name+'/')
trainer.save_model('./'+folder_name+'/')

### Get Prediction

In [ ]:
folder_name = "analysis"
tokenizer = AutoTokenizer.from_pretrained('./'+folder_name+'/')
loaded_model = AutoModelForSequenceClassification.from_pretrained('./'+folder_name+'/')

In [ ]:
input_text = "This is good. I can afford it"

# Tokenize input text
inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=512)

# Make predictions
with torch.no_grad():
    outputs = loaded_model(**inputs)
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
# Interpret the prediction
sentiment_classes = classes
predicted_sentiment = sentiment_classes[predicted_class]
print(f"The predicted sentiment of the text is: {predicted_sentiment}")

The predicted sentiment of the text is: Positive
